## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
!pip install kenlm

     |████████████████████████████████| 1.4 MB 862 kB/s 
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python3.7 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-_sn1b_8e/kenlm_33feb0ab709b4e009311494d5788ce9b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-_sn1b_8e/kenlm_33feb0ab709b4e009311494d5788ce9b/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-g6adg7vm
       cwd: /tmp/pip-install-_sn1b_8e/kenlm_33feb0ab709b4e009311494d5788ce9b/
  Complete output (269 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'kenlm' extension
  creating build/temp.linux-x86_64-3.7
  creating build/temp.linux-x86_64-3.7/util
  creating build/temp.linux-x86_64-3.7/lm
  creating build/temp.linux-x86_64-3.7/util/double-conversion
  creating build/temp.linux-x86_64-3.7/py

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...
remote: Enumerating objects: 28242, done.
remote: Counting objects: 100% (654/654), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 28242 (delta 348), reused 520 (delta 267), pack-reused 27588
Receiving objects: 100% (28242/28242), 11.90 MiB | 13.93 MiB/s, done.
Resolving deltas: 100% (21171/21171), done.


In [5]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [6]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [7]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
!python setup.py install
%cd /tmp

In [8]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [9]:
%cd /tmp/fairseq/

/tmp/fairseq


In [10]:
%%capture
!python setup.py install

In [11]:
%cd /tmp/fairseq/

/tmp/fairseq


In [12]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [13]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [14]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [15]:
!bash rungan.sh

[2021-05-31 13:54:53,404][HYDRA] Launching 5 jobs locally
[2021-05-31 13:54:53,404][HYDRA] 	#0 : task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=False checkpoint.save_dir=/kaggle/working common.seed=0
[2021-05-31 13:54:55,664][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_